In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#Random Number Generator
rng = np.random.default_rng()

In [24]:
class Player:
    def __init__(self, actions:int, strategy:np.ndarray, size:float = 1, speed:float = 1) -> None:
        self.pastPayoffs = np.empty(shape=(0,))
        self.actions = actions
        assert(strategy.ndim == 1)
        assert(strategy.shape[0] == actions)
        assert(np.sum(strategy) == 1)
        self.strategy = strategy
        self.size = size
        self.speed = speed

    def GetAction(self) -> int:
        return rng.choice(self.actions, size=1, p=self.strategy)

    def ExtendPayoffs(self, newPayoffs: np.ndarray) -> None:
        self.pastPayoffs = np.concatenate([self.pastPayoffs, newPayoffs])

    def GetPayoff(self) -> np.ndarray:
        return self.pastPayoffs
    
    def SetSpeed(self, speed:float) -> None:
        self.speed = speed
    
    def SetSize(self, size:float) -> None:
        self.size = size

    def UpdateSpeed(self, delSpeed:float) -> None:
        self.speed += delSpeed

    def UpdateSize(self, delSize:float) -> None:
        self.size += delSize

In [103]:
class Game:
    """ 
    utility: np.ndarray with dimensions nPlayers+1 and shape[i]=player[i].actions and shape[-1]=nPlayers
    """
    def __init__(self, nPlayers:int, strategies:list[np.ndarray], utility:np.ndarray):
        assert(len(strategies) == nPlayers)
        maxActions=0;
        for i in range(nPlayers):
            assert(utility.shape[i] == strategies[i].size)
            maxActions = max(maxActions, strategies[i].size)
        assert(utility.shape[-1] == nPlayers)
        self.Players = [Player(strategies[i].size, strategies[i]) for i in range(nPlayers)]
        self.utility = utility
        strategylengths = [strategies[i].size for i in range(nPlayers)]
        self.tiledProbability = np.ones(shape = [nPlayers, *strategylengths])
        self.allPayoffs = np.empty(shape = [nPlayers, 0])
        self.newPayoffs = np.empty(shape = [nPlayers, 0])
        for i in range(nPlayers):
            x = np.reshape(strategies[i], [1 if j!=i else -1 for j in range(nPlayers)])
            self.tiledProbability[i] *= x

        self.probability = np.prod(self.tiledProbability, axis=0)


    """
    returns payoff of ith player if 0<=i<nPlayers
    return an array of payoff of all players otherwise
    """
    def GetExpectedPayoff(self, i:int) -> np.ndarray:
        u = self.utility.copy()
        u *= np.expand_dims(self.probability, axis=-1)
        payoff=np.sum(u, axis=tuple(range(len(self.Players))))
        if(0<=i and i<len(self.Players)):
            return np.round(payoff[i], 8)
        else:
            return payoff
        
    def GetRandomStrategyProfile(self):
        flat_prob = self.probability.flatten()
        flat_ind = np.random.choice(len(flat_prob), p=flat_prob)
        return np.unravel_index(flat_ind, self.probability.shape)
    
    def GetPayoff(self, actions:tuple, i:int):
        p = self.utility[actions]
        if(0<=i and i<len(self.Players)):
            return p[i]
        else:
            return p

    def PlayGame(self):
        strategyProfile = self.GetRandomStrategyProfile()
        payoffs = self.GetPayoff(strategyProfile, -1)
        payoffs2 = payoffs.reshape((-1,1))
        self.newPayoffs = np.hstack([self.newPayoffs,payoffs2] )
        return payoffs
    
    def GetPayoffForStrategy(self, player:int, strategy:np.ndarray) -> np.ndarray:
        if(np.sum(strategy) != 1 or strategy.shape != (self.Players[player].actions,)):
            raise ValueError("Not a valid strategy")
        x = np.reshape(strategy, [1 if j!=player else -1 for j in range(len(self.Players))])
        y = self.tiledProbability.copy()
        y[player]=x
        prob = np.prod(y, axis=0)
        u = self.utility.copy()
        u *= np.expand_dims(prob, axis=-1)
        payoff=np.sum(u, axis=tuple(range(len(self.Players))))
        return payoff
    
    def GetPastPayoffs(self, player:int) -> np.ndarray:
        if(self.newPayoffs.size>0):
            for i in range(len(self.Players)):
                self.Players[i].ExtendPayoffs(self.newPayoffs[i])
        self.allPayoffs = np.hstack([self.allPayoffs, self.newPayoffs])
        self.newPayoffs = np.empty(shape=(len(self.Players),0))
        if(0<=player and player<len(self.Players)):
            return self.Players[player].GetPayoff()
        else:
            return self.allPayoffs
            


        

In [104]:
nPlayers = 3
strategies = [
    np.asarray([0.2, 0.55, 0.25]),
    np.asarray([1, 0]),
    np.asarray([1, 0])
]

utility = np.asarray([
    [[[1,-1,0],[1,-1,0]],[[2,-1,1],[1,-1,0]]],
    [[[-1,3,1],[1,-1,0]],[[-2,-2,0],[1,-1,0]]],
    [[[0,5,0.5],[1,-1,0]],[[5,0,0.5],[1,-1,0]]]
])

game = Game(nPlayers, strategies, utility)